# timm
* Pytorch 이미지 모델 
* Pre-trained 된 computer vision 모델을 제공하는 라이브러리 
* 허깅페이스와 유사, but NLP 대신에 컴퓨터 비전 모델을 제공한다는 차이점이 있다. 

#1. Data

In [1]:
# 우선 레포지토리를 clone (추후 새로운 모델이 timm에 추가될 때 자동으로 GItHub 반영)
! git clone --depth 1 https://github.com/rwightman/pytorch-image-models.git
%cd pytorch-image-models/results

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 532 (delta 221), reused 341 (delta 156), pack-reused 0
Receiving objects: 100% (532/532), 1.30 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/content/pytorch-image-models/results


In [6]:
# Read csv files using pandas and check the data
import pandas as pd
df_results = pd.read_csv('results-imagenet.csv')
df_results.head()

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation
0,eva_giant_patch14_560.m30m_ft_in22k_in1k,89.796,10.204,98.992,1.008,"1,014.45",560,1.0,bicubic
1,eva_giant_patch14_336.m30m_ft_in22k_in1k,89.568,10.432,98.952,1.048,"1,013.01",336,1.0,bicubic
2,eva_giant_patch14_336.clip_ft_in1k,89.476,10.524,98.824,1.176,"1,013.01",336,1.0,bicubic
3,eva_large_patch14_336.in22k_ft_in22k_in1k,89.204,10.796,98.850,1.150,304.53,336,1.0,bicubic
4,eva_giant_patch14_224.clip_ft_in1k,89.100,10.900,98.716,1.284,"1,012.56",224,1.0,bicubic


In [8]:
# get_data 함수생성 
def get_data(part, col):

    # csv를 읽어온다.
    df = pd.read_csv(f'benchmark-{part}-amp-nhwc-pt111-cu113-rtx3090.csv').merge(df_results, on='model')
    df['secs'] = 1. / df[col]

    # Family feature를 추가해서 비슷한 애들을 묶을 수 있도록 함
    df['family'] = df.model.str.extract('^([a-z]+?(?:v2)?)(?:\d|_|$)')
    df = df[~df.model.str.endswith('gn')]

    df.loc[df.model.str.contains('in22'),'family'] = df.loc[df.model.str.contains('in22'),'family'] + '_in22'
    df.loc[df.model.str.contains('resnet.*d'),'family'] = df.loc[df.model.str.contains('resnet.*d'),'family'] + 'd'
    
    return df[df.family.str.contains('^re[sg]netd?|beit|convnext|levit|efficient|vit|vgg')]

In [10]:
# df를 print해보면 새로운 속성 생긴것을 볼 수 있다. 
df = get_data('infer', 'infer_samples_per_sec')
df

,model,infer_samples_per_sec,infer_step_time,infer_batch_size,infer_img_size,param_count_x,top1,top1_err,top5,top5_err,param_count_y,img_size,crop_pct,interpolation,secs,family
0,levit_128s,21485.80,47.648,1024,224,7.78,76.530,23.470,92.866,7.134,7.78,224,0.900,bicubic,0.000047,levit
1,regnetx_002,17821.98,57.446,1024,224,2.68,68.762,31.238,88.556,11.444,2.68,224,0.875,bicubic,0.000056,regnetx
2,regnety_002,16673.08,61.405,1024,224,3.16,70.252,29.748,89.540,10.460,3.16,224,0.875,bicubic,0.000060,regnety
3,levit_128,14657.83,69.849,1024,224,9.21,78.486,21.514,94.010,5.990,9.21,224,0.900,bicubic,0.000068,levit
4,regnetx_004,14440.03,70.903,1024,224,5.16,72.396,27.604,90.830,9.170,5.16,224,0.875,bicubic,0.000069,regnetx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,resnetv2_152x2_bit_teacher_384,96.99,2639.495,256,384,236.34,83.844,16.156,97.118,2.882,236.34,384,1.000,bicubic,0.010310,resnetv2
416,resnetv2_50x3_bitm,92.92,1377.530,128,448,217.32,84.014,15.986,97.124,2.876,217.32,448,1.000,bilinear,0.010762,resnetv2
421,resnetv2_152x2_bitm,70.34,2729.533,192,448,236.34,84.510,15.490,97.432,2.568,236.34,448,1.000,bilinear,0.014217,resnetv2
422,resnetv2_101x3_bitm,57.82,2213.862,128,448,387.93,84.440,15.560,97.382,2.618,387.93,448,1.000,bilinear,0.017295,resnetv2


In [11]:
import plotly.express as px
w,h = 1000,800

def show_all(df, title, size):
    return px.scatter(df, width=w, height=h, size=df[size]**2, title=title,
        x='secs',  y='top1', log_x=True, color='family', hover_name='model', hover_data=[size])
show_all(df, 'Inference', 'infer_img_size')

* X축: 하나의 이미지를 처리하는데 걸리는 시간 (로그 스케일)  
* y축: imagenet의 정확도 
* 점의 크기: 테스트에 사용되는 이미지의 크기에 비례 
* 색깔: family classification (어느 family에 속하는지) 
* 점에 커서를 대면 정보를 확인할 수 있음.

In [12]:
# Family의 수가 너무 많기 때문에 그림에서 잘 보이는 애들만 부분집합으로 선택 
subs = 'levit|resnetd?|regnetx|vgg|convnext.*|efficientnetv2|beit'

In [15]:
def show_subs(df, title, size):
    df_subs = df[df.family.str.fullmatch(subs)]
    
    return px.scatter(df_subs, width=w, height=h, size=df_subs[size]**2, title=title,
        trendline="ols", trendline_options={'log_x':True},
        x='secs',  y='top1', log_x=True, color='family', hover_name='model', hover_data=[size])


In [16]:
show_subs(df, 'Inference', 'infer_img_size')

* 선형의 표현을 통해서 어떻게 묶을 수 있는지를 표현 
* 물론 선형으로 표현하는 것이 완벽하지 않지만 시각화를 위한 것이 목적임.

## 2. 모델의 기능 
* levit family models 는 이미지 인식이 빠르며 정확도 또한 높다는 것을 확인할 수 있음 
* 아이디어 = CNN + 트랜스포머 
* ConvNeXt
  * 속도: 중간
  * 순수 CNN
  * But use adventage of idea of 트랜스포머 
* beit
  * 정확도: 가장 높음
  * 속도: 제일 느림
  * 학습하는 dataset 자체가 큼 
    * ex) ImageNet-21k
    * 위 데이터셋은 vit model에서도 쓰임

In [17]:
# 속도와 paramter 비교에 대한 그래프 
px.scatter(df, width=w, height=h,
    x='param_count_x',  y='secs', log_x=True, log_y=True, color='infer_img_size',
    hover_name='model', hover_data=['infer_samples_per_sec', 'family']
)

# 3.  Train

In [18]:
# Get the dataset to train
tdf = get_data('train', 'train_samples_per_sec')

In [19]:
# Repeat the same family plot 
show_all(tdf, 'Training', 'train_img_size')

In [20]:
# Family 종류가 너무 많기 때문에 위에 만든 show_subs로 family개수 줄여서 봄 
show_subs(tdf, 'Training', 'train_img_size')

+ 속도은 Hardware에 따라 다를 수 있음 . 